## SQL 쿼리문

In [ ]:
-- codeit_고급_협업 일지 6 참고 (EDA 했을 때 사용한 쿼리 중 한 파일)
-- 구매 기록 테이블
-- 컬럼 정보 : `id`(PK, NN, AI), `productid`(NN), `phone_type`(NN), `created_at`(datetime), `user_id`(NN)

SELECT * FROM final.accounts_paymenthistory;

-- -------------------------------------------------

-- 테이블 데이터 행 개수 (95,140)

-- SELECT COUNT(DISTINCT id) AS table_data_count
-- FROM final.accounts_paymenthistory;

-- -------------------------------------------------

-- 각 컬럼의 NULL값 존재 확인 (NULL 존재하지 않음)

-- SELECT
--     SUM(id IS NULL) AS null_id_count,
--     SUM(productId IS NULL) AS null_productId_count,
--     SUM(phone_type IS NULL) AS null_phone_type_count,
--     SUM(created_at IS NULL) AS null_created_at_count,
--     SUM(user_id IS NULL) AS null_user_id_count
-- FROM final.accounts_paymenthistory;

-- -------------------------------------------------


-- productid, phone_type 종류 확인
-- heart.777
-- heart.200
-- heart.4000
-- heart.1000
-- A
-- I

-- SELECT DISTINCT phone_type
-- FROM final.accounts_paymenthistory;


-- -------------------------------------------------


-- 해당 테이블 내 고유 유저 수 (59,192명)

-- SELECT COUNT(DISTINCT user_id) AS user_count
-- FROM final.accounts_paymenthistory;


-- -------------------------------------------------


-- 언제부터 수집하고 수집됐는지
-- 최초 로그 기록 : 2023-05-13 21:28:34
-- 최후 로그 기록 : 2024-05-08 14:12:45

-- SELECT
-- 	MIN(created_at) AS first_created_at,
--     MAX(created_at) AS final_created_at
-- FROM final.accounts_paymenthistory;


-- -------------------------------------------------


-- productid별 몇 개 팔리고 비율이 얼마나 되는지
-- 1위 : heart777 / 57,873 / 60.83%
-- 2위 : heart1000 / 19,309 / 20.30%
-- 3위 : heart200 / 15,822 / 16.63%
-- 4위 : heart.4000 / 2,136 / 2.25%
-- 팔린 합계 95,140 == 테이블 데이터 행 개수 (95,140)

-- SELECT
-- 	productid,
--     COUNT(*) AS sales_count, -- 각 상품의 판매 수
--     ROUND(
-- 		COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2
-- 	) AS sales_ratio_pct     -- 전체 대비 비율(%)
-- FROM final.accounts_paymenthistory
-- GROUP BY productid
-- ORDER BY sales_count DESC;


-- -------------------------------------------------


-- 결제가 이뤄진 phone_type 비율
-- I(Iphone) / 61,632 / 64.78
-- A(Android) / 33,508 / 35.22

-- SELECT
-- 	phone_type,
--     COUNT(*) AS phone_count,
--     ROUND(
-- 		COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2
-- 	) AS type_ratio
-- FROM final.accounts_paymenthistory
-- group by phone_type
-- ORDER BY phone_count DESC;


-- -------------------------------------------------


-- 한 번만 결제한 유저 특징
-- 전체 59,192명 중  43,049명(72.73%)
-- iphone (26,525명 / 61.62%)
-- android (16,524명 / 38.38%)


-- WITH user_pay AS (
--   SELECT user_id, COUNT(*) AS pay_count
--   FROM final.accounts_paymenthistory
--   GROUP BY user_id
-- )
-- SELECT
--   SUM(pay_count = 1)                           AS one_time_users,         -- 1회 결제 유저 수
--   COUNT(*)                                     AS total_unique_users,     -- 결제 이력 보유 고유 유저 수 (예상: 59,192)
--   ROUND(SUM(pay_count = 1) * 100.0 / COUNT(*), 2) AS one_time_ratio_pct   -- 비율(%)
-- FROM user_pay;



-- 한 번만 결제한 유저들의 phone_type 비율
-- WITH user_pay AS (
--     SELECT user_id, COUNT(*) AS pay_count
--     FROM final.accounts_paymenthistory
--     GROUP BY user_id
--     HAVING COUNT(*) = 1              -- 한 번만 결제한 유저만 필터링
-- )
-- SELECT
--     p.phone_type,
--     COUNT(DISTINCT p.user_id) AS user_count,   -- phone_type별 유저 수
--     ROUND(
--         COUNT(DISTINCT p.user_id) * 100.0 / SUM(COUNT(DISTINCT p.user_id)) OVER(),
--         2
--     ) AS phone_ratio_pct                        -- 전체 대비 비율(%)
-- FROM final.accounts_paymenthistory AS p
-- JOIN user_pay AS u
--     ON p.user_id = u.user_id
-- GROUP BY p.phone_type
-- ORDER BY user_count DESC;


-- -------------------------------------------------


-- 2회 이상 구매한 유저 특징
-- 전체 59,192명 중 16,143명 27.27%
-- iphone (10,778명 / 66.53%)
-- android (5,421명 / 33.47%)

-- 2회 이상 결제한 고유 유저 수 및 비율
-- WITH user_pay AS (
--   SELECT user_id, COUNT(*) AS pay_count
--   FROM final.accounts_paymenthistory
--   GROUP BY user_id
-- )
-- SELECT
--   SUM(pay_count >= 2) AS repeat_users,                         -- 2회 이상 결제 유저 수
--   COUNT(*) AS total_users,                                     -- 전체 결제 유저 수
--   ROUND(SUM(pay_count >= 2) * 100.0 / COUNT(*), 2) AS repeat_ratio_pct -- 비율(%)
-- FROM user_pay;


-- -- 2회 이상 결제한 유저들의 phone_type 비율
-- WITH user_pay AS (
--   SELECT user_id, COUNT(*) AS pay_count
--   FROM final.accounts_paymenthistory
--   GROUP BY user_id
--   HAVING COUNT(*) >= 2
-- )
-- SELECT
--     p.phone_type,
--     COUNT(DISTINCT p.user_id) AS user_count,
--     ROUND(
--         COUNT(DISTINCT p.user_id) * 100.0 / SUM(COUNT(DISTINCT p.user_id)) OVER(), 2
--     ) AS phone_ratio_pct
-- FROM final.accounts_paymenthistory AS p
-- JOIN user_pay AS u
--   ON p.user_id = u.user_id
-- GROUP BY p.phone_type
-- ORDER BY user_count DESC;


-- -------------------------------------------------


-- 리텐션
-- 테이블 데이터 내 2회 이상 구매 유저의 두 번째 결제까지 걸린 평균 시간 (일 단위)
-- 2회 이상 결제 유저 : 16,143명
-- 평균 소요 일수 : 4.57일
-- 최소 결제 소요 일수 : 0일(즉시 재결재)
-- 최대 결제 소유 일수 : 360일

-- WITH ordered_payments AS (
--     SELECT
--         user_id,
--         created_at,
--         ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at) AS rn
--     FROM final.accounts_paymenthistory
-- ),
-- two_purchases AS (
--     SELECT
--         a.user_id,
--         TIMESTAMPDIFF(DAY, a.created_at, b.created_at) AS days_to_second
--     FROM ordered_payments AS a
--     JOIN ordered_payments AS b
--       ON a.user_id = b.user_id
--      AND a.rn = 1           -- 첫 번째 결제
--      AND b.rn = 2           -- 두 번째 결제
-- )
-- SELECT
--     COUNT(*) AS users_with_2plus_purchases,                -- 2회 이상 결제 유저 수
--     ROUND(AVG(days_to_second), 2) AS avg_days_to_second,   -- 평균 걸린 일수
--     MIN(days_to_second) AS min_days,
--     MAX(days_to_second) AS max_days
-- FROM two_purchases;


-- -------------------------------------------------

-- 매출 1위 vs 수익 1위
-- qty(판매량)
-- revenue(매출액)
-- | productid  |  단가(₩) | 비고             |
-- | heart.200  |  2,700 | 신규 유입용, 엔트리 상품 |
-- | heart.777  |  5,500 | 주력 상품          |
-- | heart.1000 | 13,000 | 프리미엄 상품        |
-- | heart.4000 | 49,950 | 헤비유저용 고액 상품    |

-- | productid  |   판매수(qty) |  단가(₩) |          매출(₩) |   매출 비중(%) |
-- | heart.777  |     57,873 |  3,500 |     20,255,550 |      59.20 |
-- | heart.1000 |     19,309 |  4,500 |      8,689,050 |      25.40 |
-- | heart.4000 |      2,136 | 18,000 |      3,848,000 |      11.24 |
-- | heart.200  |     15,822 |    900 |      1,429,800 |       4.18 |
-- |   합계       |   95,140  |     —  |   34,222,850   |   100.00    |




-- WITH priced AS (
--   SELECT *,
--     CASE productid
--       WHEN 'heart.200'  THEN 900   -- 900원 가정
--       WHEN 'heart.777'  THEN 3500  -- 3,500원 가정
--       WHEN 'heart.1000' THEN 4500  -- 4,500원 가정
--       WHEN 'heart.4000' THEN 18000  -- 18,000원 가정
--     END AS price
--   FROM final.accounts_paymenthistory
-- )
-- SELECT
--   productid,
--   COUNT(*) AS qty,
--   SUM(price) AS revenue,
--   ROUND(SUM(price)*100.0/SUM(SUM(price)) OVER (),2) AS revenue_ratio_pct
-- FROM priced
-- GROUP BY productid
-- ORDER BY revenue DESC;

In [ ]:
-- codeit_고급_협업 일지 6 참고 (역시 EDA 쿼리 중 한 파일 내용)
-- 헥클 이벤트 발생 목록 테이블 조사
-- 해당 테이블의 수집 기간은 약 4주라는 특징이 있음
-- 컬럼 정보 : `event_id`(VARCHAR, PK, NN), `event_datetime`(DATETIME), `event_key`(VARCHAR), `session_id`(VARCHAR), `id`(VARCHAR),
-- `item_name`(VARCHAR), `page_name`(VARCHAR), `friend_count`(INT), `votes_count`(INT), `heart_balance`(INT), `question_id`(INT)


-- 테이블 데이터 행 개수 (11,441,319)

-- SELECT COUNT(DISTINCT event_id) AS table_event_id_count
-- FROM final.hackle_events;

-- --------------------------------------------------------------------------------


-- 이벤트 종류 확인
-- SELECT
--     DISTINCT event_key
-- FROM final.hackle_events;

-- 유저 이벤트 정리
-- | 이벤트명                               | 의미 / 발생 시점                                           |
-- | 1. $session_start                    | 앱 실행 또는 세션이 시작될 때 자동 기록됨 (유저가 앱에 진입한 시점)      | X
-- | 2. $session_end                      | 유저 세션이 종료될 때 기록됨 (앱 종료, 백그라운드 이동 등)           | X
-- | 3. launch_app                        | 앱이 실행된 순간 (앱 런치) — `$session_start`와 거의 동일 맥락 | X
-- | 4. complete_signup                   | 회원가입 절차가 완료된 시점                               |
-- | 5. view_signup                       | 회원가입 화면(뷰)이 표시될 때                             |
-- | 6. view_login                        | 로그인 화면 진입 시점                                  |
-- | 7. complete_purchase                 | 결제(상품 구매)가 완료된 시점                             |
-- | 8. click_purchase                    | 구매 버튼 클릭 이벤트                                  |
-- | 9. view_shop                         | 상점(하트샵, 결제페이지 등) 화면을 열었을 때                    |
-- | 10. complete_question                 | 질문이 정상적으로 완료(제출)된 시점                          |
-- | 11. click_question_start              | “질문하기 시작” 버튼 클릭 시                             |
-- | 12. click_question_ask                | 질문 작성 화면 진입 혹은 질문 작성 버튼 클릭                  |
-- | 13. click_question_open               | 질문 상세 화면 열기                                   |
-- | 14. click_question_share              | 질문 공유 버튼 클릭                                   |
-- | 15. skip_question                     | 질문 넘기기(스킵) 클릭 시                               | X
-- | 16. click_random_ask_normal           | 랜덤 질문(일반 모드) 요청 클릭                            |
-- | 17. click_random_ask_other            | 다른 유저에게 랜덤 질문 보내기 클릭                          |
-- | 18. click_random_ask_shuffle          | 랜덤 질문 섞기(셔플) 기능 클릭                            |
-- | 19. click_timeline_chat_start         | 타임라인에서 채팅 시작 버튼 클릭                            |
-- | 20. click_appbar_chat_rooms           | 상단 앱바의 “채팅방” 버튼 클릭                            |
-- | 21. click_appbar_alarm_center         | 상단 알림 센터 클릭                                   |
-- | 22. click_appbar_friend_plus          | 상단 친구 추가 버튼 클릭                                |
-- | 23. click_appbar_setting              | 상단 설정 버튼 클릭                                   |
-- | 24. click_attendance                  | 출석체크(출석 이벤트 버튼) 클릭                            |
-- | 25. click_autoadd_contact             | 연락처 자동 추가 기능 실행                               |
-- | 26. click_friend_invite               | 친구 초대 버튼 클릭                                   |
-- | 27. click_invite_friend               | 친구 초대 팝업 혹은 링크 공유 클릭                          |
-- | 28. click_bottom_navigation_lab       | 하단 네비게이션에서 “랩(Lab)” 탭 클릭                      |
-- | 29. click_bottom_navigation_profile   | 하단 네비게이션에서 “프로필” 탭 클릭                         |
-- | 30. click_bottom_navigation_questions | 하단 네비게이션에서 “질문” 탭 클릭                          |
-- | 31. click_bottom_navigation_timeline  | 하단 네비게이션에서 “타임라인” 탭 클릭                        |
-- | 32. click_community_chat              | 커뮤니티 채팅방 진입 클릭                                |
-- | 33. click_copy_profile_link_ask       | 프로필에서 “질문 링크 복사” 클릭                           |
-- | 34. click_copy_profile_link_profile   | 프로필 화면에서 “프로필 링크 복사” 클릭                       |
-- | 35. click_notice                      | 공지사항 리스트 진입 클릭                                | X
-- | 36. click_notice_detail               | 공지사항 상세 클릭                                    | X
-- | 37. click_profile_ask                 | 프로필 내에서 질문하기 클릭                               |
-- | 38. view_home_tap                     | 홈 화면 진입(홈 탭 클릭)                               |
-- | 39. view_profile_tap                  | 프로필 탭 진입                                      |
-- | 40. view_friendplus_tap               | 친구 추가 탭 진입                                    |
-- | 41. view_questions_tap                | 질문 탭 진입                                       |
-- | 42. view_lab_tap                      | 실험실(랩) 탭 진입                                   | ▲
-- | 43. view_timeline_tap                 | 타임라인 탭 진입                                     |
-- | 44. button                            | 일반적인 버튼 클릭 이벤트 (세부 카테고리 지정 안 된 클릭 시 기록)  | X


-- --------------------------------------------------------------------------------


-- 각 컬럼별 결측 수 확인
-- event_id : NULL 없음
-- event_datetime : NULL 없음
-- event_key : NULL 없음
-- session_id : NULL 없음
-- id : NULL 없음
-- item_name : NULL 없음
-- page_name : NULL 없음
-- friend_count : NULL 7,525,56건 (약 65.7%)
-- votes_count : NULL 7,545,54건 (약 65.9%)
-- heart_balance : NULL 7,286,43건 (약 63.7%)
-- question_id : NULL 10,991,835건 (약 96.1%) 스킵한 질문 ID

-- SELECT
--     SUM(event_id IS NULL) AS null_event_id_count,
--     SUM(event_datetime IS NULL) AS null_event_datetime_list_count,
--     SUM(event_key IS NULL) AS null_event_key_count,
--     SUM(session_id IS NULL) AS null_session_id_count,
--     SUM(id IS NULL) AS null_id_count,
--     SUM(item_name IS NULL) AS null_item_name_count,
--     SUM(page_name IS NULL) AS null_page_name_count,
--     SUM(friend_count IS NULL) AS null_friend_count_count,
--     SUM(votes_count IS NULL) AS null_votes_count_count,
--     SUM(heart_balance IS NULL) AS null_heart_balance_count,
-- 	SUM(question_id IS NULL) AS null_question_id_count
-- FROM final.hackle_events;


-- --------------------------------------------------------------------------------


-- 이벤트들 중에서 가장 많이 발생한 이벤트 확인

-- | 순위 | 이벤트명 (`event_key`)              | 발생 횟수  | 주요 의미 / 발생 맥락          |
-- | 1  | view_lab_tap                      | 1,266,665 | 실험실(Lab) 탭 진입            |  → (2)홈→질문 피그마 탭의 "시작 가능" 하단 화면의 `질문 추천` 클릭 시 진입되는 화면 진입 시 & "질문 대기" 화면의 `재밌는 질문 아이디어가 있어?` 버튼 클릭 시 진입되는 화면 진입 시
-- | 2  | view_timeline_tap                 | 1,194,508 | 타임라인 탭 진입               |  → 타임라임 화면 진입
-- | 3  | $session_start                    | 1,036,852 | 앱 세션 시작 (앱 실행/복귀)     | → 단순 앱 시작
-- | 4  | launch_app                        |   986,388 | 앱 실행 시점                   | → 단순 앱 실행 시 동시에 출력
-- | 5  | click_question_open               |   816,801 | 질문 상세 화면 열기            | → (3)Ping 피그마 탭의 "PING" 화면의 임의의 질문 클릭 (클릭 시 (3)PING 피그마탭의 "상세" 화면으로 이동하는 것으로 추정)
-- | 6  | click_bottom_navigation_questions |   769,163 | 하단 “질문” 탭 클릭            | → ???
-- | 7  | click_bottom_navigation_profile   |   653,507 | 하단 “프로필” 탭 클릭          | → ???
-- | 8  | $session_end                      |   649,658 | 앱 세션 종료 (백그라운드 등)   | → 단순 앱 종료
-- | 9  | click_bottom_navigation_timeline  |   536,051 | 하단 “타임라인” 탭 클릭        | → ???
-- | 10 | skip_question                     |   454,981 | 질문 스킵(넘기기) 클릭         | → (2)홈→질문 피그마 탭의 "질문" 창의 `건너뛰기` 버튼 클릭
-- | 11 | click_bottom_navigation_lab       |   453,683 | 하단 “랩” 탭 클릭             | → ???
-- | 12 | view_profile_tap                  |   413,294 | 프로필 탭 진입                | → (6) 프로필 피그마 탭의 "내 프로필" 화면 진입
-- | 13 | view_questions_tap                |   353,400 | 질문 탭 진입                  | → (2)홈→질문 피그마 탭의 "질문" 화면 진입 (`click_question_start` 이후에 나오는 로그일 가능성)
-- | 14 | click_appbar_alarm_center         |   253,541 | 상단 알림 센터 진입           | → (2)홈→질문 피그마 탭의 "시작 가능" 화면의 좌측 상단 `PING` 버튼 클릭 (클릭 후 (3)PING 피그마 탭의 "PING" 화면으로 이동하는 것으로 추정)
-- | 15 | click_notice_detail               |   229,358 | 공지사항 상세 보기            | → (2)홈→질문 피그마 탭의 "시작 가능" 화면의 상단 `확성기 배너` 클릭
-- | 16 | click_question_start              |   220,385 | 질문 시작 버튼 클릭           | → (2)홈→질문 피그마 탭의 "시작 가능" 화면의 `START` 버튼 클릭
-- | 17 | click_random_ask_shuffle          |   184,217 | 랜덤 질문 섞기 클릭           | → ??? (2)홈→질문 피그마 탭의 "질문" 화면에 `이름 셔플` 버튼 외 셔플 버튼이 보이지 않음
-- | 18 | click_attendance                  |   157,737 | 출석 이벤트 클릭              | → (2)홈→질문 피그마 탭의 "시작 가능" 화면에 우측 하단 `출석체크` 버튼 클릭
-- | 19 | complete_question                 |   154,105 | 질문 완료(제출)               | → (2)홈→질문 피그마 탭의 "완료" 화면 출력과 동시에 생성되는 것으로 추정
-- | 20 | click_appbar_chat_rooms           |   148,422 | 채팅방 보기                   | → (3)PING 피그마 탭의 "1.6.0 업데이트 <채팅 추가>" 화면에서 3 번째 화면 하단 `채팅방 열기 ♥200` 버튼 클릭
-- | 21 | click_question_ask                |   136,766 | 질문 물어보기 클릭            | → (3)PING 피그마 탭의 "상세" 화면의 `핑에게 답장하기` 버튼 클릭? ▲
-- | 22 | click_question_share              |    69,241 | 질문 공유 클릭                | → (3)PING 피그마 탭의 "답장 완료" 화면의 `공유하기` 버튼 클릭? ▲
-- | 23 | click_timeline_chat_start         |    50,186 | 타임라인 채팅 시작 클릭        | → ???
-- | 24 | click_appbar_friend_plus          |    49,545 | 친구 추가 버튼 클릭            | → (8)친구+ 피그마 탭의 "친구+" 화면의 `친구추가` 버튼 클릭
-- | 25 | view_login                        |    49,275 | 로그인 화면 진입              | → (1)온보딩 피그마 탭의 "기존 계정 로그인 인증" 좌측 화면 진입
-- | 26 | click_appbar_setting              |    26,685 | 설정 버튼 클릭                | → (6)프로필 피그마 탭의 "내 프로필" 화면의 우측 상단 `톱니바퀴` 버튼 클릭? ▲
-- | 27 | view_shop                         |    26,607 | 상점(하트샵) 화면 진입         | → (7)하트 충전소 피그마 탭의 "하트 충전소" 좌측 화면 진입
-- | 28 | view_signup                       |    25,630 | 회원가입 화면 진입            | → (1)온보딩 피그마 탭의 "학교 선택" 좌측 화면 진입
-- | 29 | click_random_ask_normal           |    18,653 | 랜덤 질문(일반모드) 요청       | → ???
-- | 30 | click_profile_ask                 |    14,627 | 프로필에서 질문하기 클릭       | → 프로필 하단에 자주 걸리는 질문 top3 그 질문인가???? 아직 애매함
-- | 31 | click_purchase                    |    13,039 | 구매 버튼 클릭                | → (7)하트 충전소 피그마 탭의 "하트 충전소" 좌측 화면 아이템들 중 하나 클릭 후 아마도 구매하기 버튼 나오면 그거 눌렀을때??? ▲
-- | 32 | click_random_ask_other            |    11,075 | 다른 유저에게 랜덤 질문 보내기  | → ???
-- | 33 | view_home_tap                     |     5,392 | 홈 탭 진입                    | → (2)홈→질문 피그마 탭의 "시작 가능" 화면 진입
-- | 34 | click_friend_invite               |     3,221 | 친구 초대 버튼 클릭            | → (8)친구+ 피그마 탭의 "친구+" 화면의 초대 가능한 친구 목록에서 `초대하기` 버튼 클릭
-- | 35 | complete_purchase                 |     2,201 | 결제 완료                     | → 피그마에서 결제 완료 화면이 나오지 않지만, click_purchase 해당 로그 등장 후 완료 시 해당 로그 발생으로 추정 ▲
-- | 36 | click_copy_profile_link_profile   |     1,703 | 프로필 링크 복사 클릭          | → (6)프로필 피그마 탭의 "내 프로필" 화면에서 `프로필 링크` 버튼 클릭
-- | 37 | click_community_chat              |     1,335 | 커뮤니티 채팅방 진입           | → (4)(메시지 (1.6.0 업데이트 추가) 피그마 탭의 "상대가 누군지 모르는 경우" 화면 진입 시  ▲
-- | 38 | click_invite_friend               |     1,054 | 친구 초대 팝업/링크 공유       | → (2)홈→질문 피그마 탭의 "실행 불가 - 친구 수가 적음" & "실행 불가 - 같은 학교 유저가 40명 미만" / (3)PING "받은 PING이 없는 경우" 화면의 `초대 링크 복사` 버튼 클릭
-- | 39 | complete_signup                   |       974 | 회원가입 완료                 | → (1)온보딩 피그마 탭의 "알림 권한 등록" 화면 진입 시
-- | 40 | click_autoadd_contact             |       918 | 연락처 자동 추가 버튼 클릭     | → (1)온보딩 피그마 탭의 "자동 친구 추가" 화면의 `자동으로 친구추가` 버튼 클릭
-- | 41 | button                            |       428 | 일반 버튼 클릭(비분류)         | → ????
-- | 42 | click_copy_profile_link_ask       |        40 | 프로필 링크 복사???? 클릭?     | → ????
-- | 43 | view_friendplus_tap               |         7 | 친구추가 탭 진입               | → (8)친구+ 피그마 탭의 "친구+" 화면 진입
-- | 44 | click_notice                      |         1 | 공지사항 리스트 진입 클릭       | → ??? (2)홈→질문 피그마 탭의 "시작 가능" 화면의 상단 `확성기 배너` 클릭 했을 때 리스트가 나오는건지? 그렇다고 보기엔 횟수가 너무 적어서 상단 다른 이벤트에 같은 뜻으로 해석한 이벤트 값이 있음


-- SELECT
--     RANK() OVER (ORDER BY COUNT(*) DESC) AS `rank`,  -- 발생 횟수 순위
--     event_key,
--     COUNT(*) AS event_count
-- FROM final.hackle_events
-- GROUP BY event_key
-- ORDER BY event_count DESC;


-- ------------------------------------------------------------------------------

-- 고유 `session_id` 별 `event_key` 등장 횟수 확인
-- SELECT
-- 	event_key,
--     COUNT(DISTINCT session_id) AS `unique_count`
-- FROM final.hackle_events
-- GROUP BY event_key
-- ORDER BY unique_count DESC;


-- |  순위 | 이벤트명 (`event_key`)                | 고유 유저 수 (`unique_user_count`) |
-- |  1  | `$session_start`                    |                       252,877 |
-- |  2  | `launch_app`                        |                       251,179 |
-- |  3  | `$session_end`                      |                       201,549 |
-- |  4  | `view_lab_tap`                      |                       177,248 |
-- |  5  | `click_bottom_navigation_questions` |                       171,524 |
-- |  6  | `view_timeline_tap`                 |                       166,495 |
-- |  7  | `click_bottom_navigation_profile`   |                       164,791 |
-- |  8  | `click_bottom_navigation_timeline`  |                       150,463 |
-- |  9  | `click_bottom_navigation_lab`       |                       142,733 |
-- |  10 | `click_appbar_alarm_center`         |                       140,284 |
-- |  11 | `view_profile_tap`                  |                       121,241 |
-- |  12 | `click_question_open`               |                       113,699 | 3 (view_login, view_home_tap 이벤트 키 둘 중 하나를 보유중인 유저들 중에서)
-- |  13 | `view_questions_tap`                |                       107,717 |
-- |  14 | `click_notice_detail`               |                        95,541 |
-- |  15 | `click_question_start`              |                        75,875 | 2
-- |  16 | `click_question_ask`                |                        72,351 |
-- |  17 | `click_appbar_chat_rooms`           |                        61,769 |
-- |  18 | `complete_question`                 |                        50,013 |
-- |  19 | `click_attendance`                  |                        47,834 |
-- |  20 | `skip_question`                     |                        39,423 |
-- |  21 | `view_login`                        |                        37,016 |  1
-- |  22 | `click_appbar_friend_plus`          |                        25,691 |
-- |  23 | `click_timeline_chat_start`         |                        25,651 |
-- |  24 | `click_appbar_setting`              |                        19,349 |
-- |  25 | `view_shop`                         |                        17,488 |
-- |  26 | `click_random_ask_normal`           |                        11,898 |
-- |  27 | `click_profile_ask`                 |                        10,307 |
-- |  28 | `click_question_share`              |                        10,283 |
-- |  29 | `click_random_ask_other`            |                         7,845 |
-- |  30 | `view_signup`                       |                         7,614 |
-- |  31 | `click_random_ask_shuffle`          |                         6,704 |
-- |  32 | `click_purchase`                    |                         6,487 |
-- |  33 | `view_home_tap`                     |                         4,540 |  1
-- |  34 | `click_friend_invite`               |                         2,118 |
-- |  35 | `complete_purchase`                 |                         1,638 |  4
-- |  36 | `click_copy_profile_link_profile`   |                         1,558 |
-- |  37 | `complete_signup`                   |                           973 |
-- |  38 | `click_community_chat`              |                           807 |
-- |  39 | `click_autoadd_contact`             |                           746 |
-- |  40 | `click_invite_friend`               |                           653 |
-- |  41 | `button`                            |                           305 |
-- |  42 | `click_copy_profile_link_ask`       |                            31 |
-- |  43 | `view_friendplus_tap`               |                             6 |
-- |  44 | `click_notice`                      |                             1 |


-- --------------------------------------------------------------------------------


-- session_id 별로 event_key 존재 (GROUP BY)


-- `session_id` 고유 데이터 수 : 253,616

-- SELECT COUNT(DISTINCT session_id)
-- FROM final.hackle_events;


-- --------------------------------------------------------------------------------


-- 전체 기간 기준(2023-07-18 ~ 2023-08-10 (약 4주))
-- 하루 평균 event_key에 찍히는 고유 session_id 수 확인
-- | 순위 | 이벤트명 (`event_key`)           | 고유 세션 수 | 일평균 세션 수 |
-- |  1 | $session_start                    | 252,877 | 10,536.54 |
-- |  2 | launch_app                        | 251,179 | 10,465.79 |
-- |  3 | $session_end                      | 201,549 |  8,397.88 |
-- |  4 | view_lab_tap                      | 177,248 |  7,385.33 |
-- |  5 | click_bottom_navigation_questions | 171,524 |  7,146.83 |
-- |  6 | view_timeline_tap                 | 166,495 |  6,937.29 |
-- |  7 | click_bottom_navigation_profile   | 164,791 |  6,866.29 |
-- |  8 | click_bottom_navigation_timeline  | 150,463 |  6,269.29 |
-- |  9 | click_bottom_navigation_lab       | 142,733 |  5,947.21 |
-- | 10 | click_appbar_alarm_center         | 140,284 |  5,845.17 |
-- | 11 | view_profile_tap                  | 121,241 |  5,051.71 |
-- | 12 | click_question_open               | 113,699 |  4,737.46 |
-- | 13 | view_questions_tap                | 107,717 |  4,488.21 |
-- | 14 | click_notice_detail               |  95,541 |  3,980.88 |
-- | 15 | click_question_start              |  75,875 |  3,161.46 |
-- | 16 | click_question_ask                |  72,351 |  3,014.63 |
-- | 17 | click_appbar_chat_rooms           |  61,769 |  2,573.71 |
-- | 18 | complete_question                 |  50,013 |  2,083.88 |
-- | 19 | click_attendance                  |  47,834 |  1,993.08 |
-- | 20 | skip_question                     |  39,423 |  1,642.63 |
-- | 21 | view_login                        |  37,016 |  1,542.33 |
-- | 22 | click_appbar_friend_plus          |  25,691 |  1,070.46 |
-- | 23 | click_timeline_chat_start         |  25,651 |  1,068.79 |
-- | 24 | click_appbar_setting              |  19,349 |    806.21 |
-- | 25 | view_shop                         |  17,488 |    728.67 |
-- | 26 | click_random_ask_normal           |  11,898 |    495.75 |
-- | 27 | click_profile_ask                 |  10,307 |    429.46 |
-- | 28 | click_question_share              |  10,283 |    428.46 |
-- | 29 | click_random_ask_other            |   7,845 |    326.88 |
-- | 30 | view_signup                       |   7,614 |    317.25 |
-- | 31 | click_random_ask_shuffle          |   6,704 |    279.33 |
-- | 32 | click_purchase                    |   6,487 |    270.29 |
-- | 33 | view_home_tap                     |   4,540 |    189.17 |
-- | 34 | click_friend_invite               |   2,118 |     88.25 |
-- | 35 | complete_purchase                 |   1,638 |     68.25 |
-- | 36 | click_copy_profile_link_profile   |   1,558 |     64.92 |
-- | 37 | complete_signup                   |     973 |     40.54 |
-- | 38 | click_community_chat              |     807 |     33.63 |
-- | 39 | click_autoadd_contact             |     746 |     31.08 |
-- | 40 | click_invite_friend               |     653 |     27.21 |
-- | 41 | button                            |     305 |     12.71 |
-- | 42 | click_copy_profile_link_ask       |      31 |      1.29 |
-- | 43 | view_friendplus_tap               |       6 |      0.25 |
-- | 44 | click_notice                      |       1 |      0.04 |


-- WITH event_summary AS (
--     SELECT
--         event_key,
--         COUNT(DISTINCT session_id) AS total_unique_sessions
--     FROM final.hackle_events
--     GROUP BY event_key
-- ),
-- date_bounds AS (
--     SELECT
--         DATEDIFF(MAX(event_datetime), MIN(event_datetime)) + 1 AS total_days
--     FROM final.hackle_events
-- )
-- SELECT
--     e.event_key,
--     e.total_unique_sessions,
--     ROUND(e.total_unique_sessions / d.total_days, 2) AS avg_daily_unique_sessions
-- FROM event_summary e
-- CROSS JOIN date_bounds d
-- ORDER BY avg_daily_unique_sessions DESC;


-- --------------------------------------------------------------------------------


-- 각 고유 세션 별 이벤트 발생 시가 순으로  구분자 `,`로 설정(테이블 내 최초 ~ 최종)
-- 해당 리스트는 순서대로 되어있기 때문에, 각 session에서 각 이벤트 발생 후 다음에 발생되는 이벤트가 뭔지 조사할 수 있음
-- 그렇게 조사한 이벤트들을 "각 이벤트 발생 후 후속 이벤트로 많이 발생된 이벤트 1, 2, 3"위를 선정하여 발생 수와 같이 컬럼으로 표시
-- count_1(1위 후속 이벤트) 내림차순으로 정렬하여 유저 밀집 통행 구간 확인
-- 각 이벤트 진행 후 어떤 행동을 하는지 파악하기 위해 진행
-- ## 각 세션별 이벤트 순서를 고려해서, 모든 세션을 합산한 후
-- ## 각 이벤트의 후속 이벤트 빈도를 집계하고, 그 중 Top3를 구하는 것

-- | 순위 | 이벤트명 (`event_key`)             | 1위 후속 이벤트                       | count_1 | 2위 후속 이벤트                      | count_2 | 3위 후속 이벤트                       | count_3 |
-- |  1 | click_question_open               | click_question_open               | 479,386 | click_question_share              |  59,539 | view_timeline_tap                 |  38,053 |
-- |  2 | $session_start                    | launch_app                        | 448,181 | $session_end                      | 116,002 | $session_start                    |  56,607 |
-- |  3 | launch_app                        | $session_start                    | 423,833 | $session_end                      | 123,861 | launch_app                        |  53,731 |
-- |  4 | $session_end                      | $session_start                    | 377,535 | launch_app                        | 247,087 | view_login                        |   7,434 |
-- |  5 | skip_question                     | skip_question                     | 326,333 | complete_question                 |  99,290 | $session_end                      |   8,349 |
-- |  6 | view_timeline_tap                 | view_lab_tap                      | 296,761 | click_bottom_navigation_timeline  | 190,089 | click_bottom_navigation_questions | 130,486 |
-- |  7 | view_lab_tap                      | view_timeline_tap                 | 270,209 | click_bottom_navigation_profile   | 199,042 | click_bottom_navigation_lab       | 186,208 |
-- |  8 | click_bottom_navigation_profile   | view_lab_tap                      | 201,144 | view_profile_tap                  | 129,439 | view_timeline_tap                 |  94,503 |
-- |  9 | click_bottom_navigation_timeline  | view_timeline_tap                 | 199,485 | view_lab_tap                      |  78,108 | click_bottom_navigation_questions |  73,255 |
-- | 10 | click_bottom_navigation_lab       | view_lab_tap                      | 194,969 | click_bottom_navigation_profile   |  73,530 | view_timeline_tap                 |  69,897 |
-- | 11 | click_random_ask_shuffle          | click_random_ask_shuffle          | 170,966 | click_random_ask_normal           |   3,566 | click_random_ask_other            |   2,252 |
-- | 12 | click_appbar_alarm_center         | click_notice_detail               | 155,083 | click_bottom_navigation_questions |  10,942 | click_appbar_alarm_center         |  10,504 |
-- | 13 | click_bottom_navigation_questions | view_timeline_tap                 | 132,751 | click_bottom_navigation_questions | 123,417 | view_questions_tap                | 105,602 |
-- | 14 | view_profile_tap                  | click_bottom_navigation_profile   | 116,776 | view_lab_tap                      | 115,579 | view_timeline_tap                 |  56,869 |
-- | 15 | click_question_start              | skip_question                     | 112,135 | complete_question                 |  53,861 | $session_end                      |   9,963 |
-- | 16 | view_questions_tap                | click_bottom_navigation_questions |  98,767 | view_timeline_tap                 |  87,088 | view_lab_tap                      |  38,801 |
-- | 17 | click_notice_detail               | click_notice_detail               |  69,841 | $session_end                      |  16,473 | click_bottom_navigation_questions |  15,844 |
-- | 18 | click_question_share              | click_question_open               |  40,894 | view_timeline_tap                 |   4,735 | $session_end                      |   4,137 |
-- | 19 | click_attendance                  | $session_end                      |  35,248 | click_question_start              |  19,785 | click_question_open               |  19,374 |
-- | 20 | complete_question                 | $session_end                      |  27,810 | click_question_open               |  20,219 | view_timeline_tap                 |  17,867 |
-- | 21 | click_appbar_chat_rooms           | click_appbar_alarm_center         |  24,787 | $session_end                      |  14,864 | click_appbar_chat_rooms           |  12,234 |
-- | 22 | click_question_ask                | view_lab_tap                      |  21,281 | click_bottom_navigation_timeline  |  19,930 | click_bottom_navigation_lab       |  18,344 |
-- | 23 | view_signup                       | view_signup                       |  17,115 | view_login                        |   4,040 | $session_end                      |   2,156 |
-- | 24 | click_timeline_chat_start         | click_timeline_chat_start         |  12,794 | view_lab_tap                      |   8,598 | $session_end                      |   4,186 |
-- | 25 | view_login                        | $session_end                      |  11,965 | click_question_open               |  10,919 | $session_start                    |   7,679 |
-- | 26 | view_shop                         | click_purchase                    |   9,309 | launch_app                        |   4,831 | $session_end                      |   1,950 |
-- | 27 | click_appbar_friend_plus          | $session_end                      |   5,724 | view_lab_tap                      |   5,330 | view_timeline_tap                 |   4,246 |
-- | 28 | click_appbar_setting              | view_lab_tap                      |   5,500 | $session_end                      |   3,880 | click_bottom_navigation_questions |   2,609 |
-- | 29 | click_random_ask_normal           | click_random_ask_shuffle          |   3,793 | view_lab_tap                      |   1,945 | click_random_ask_normal           |   1,843 |
-- | 30 | click_purchase                    | click_purchase                    |   3,445 | complete_purchase                 |   2,127 | view_timeline_tap                 |     871 |
-- | 31 | click_profile_ask                 | view_lab_tap                      |   2,889 | click_profile_ask                 |   1,976 | $session_end                      |   1,201 |
-- | 32 | click_random_ask_other            | click_random_ask_shuffle          |   1,977 | click_random_ask_normal           |   1,621 | view_lab_tap                      |   1,064 |
-- | 33 | view_home_tap                     | launch_app                        |   1,660 | $session_end                      |   1,562 | $session_start                    |     964 |
-- | 34 | click_friend_invite               | click_invite_friend               |     571 | view_timeline_tap                 |     265 | $session_end                      |     255 |
-- | 35 | complete_signup                   | $session_start                    |     415 | view_timeline_tap                 |     118 | click_question_start              |      91 |
-- | 36 | complete_purchase                 | view_timeline_tap                 |     364 | view_lab_tap                      |     283 | click_question_open               |     256 |
-- | 37 | click_copy_profile_link_profile   | view_lab_tap                      |     346 | click_appbar_setting              |     323 | $session_end                      |     148 |
-- | 38 | click_community_chat              | view_timeline_tap                 |     316 | click_community_chat              |     215 | view_profile_tap                  |     171 |
-- | 39 | click_invite_friend               | click_invite_friend               |     224 | click_friend_invite               |     133 | launch_app                        |     124 |
-- | 40 | click_autoadd_contact             | view_lab_tap                      |     139 | click_bottom_navigation_profile   |      97 | click_autoadd_contact             |      78 |
-- | 41 | button                            | button                            |     118 | launch_app                        |     114 | $session_end                      |      83 |
-- | 42 | click_copy_profile_link_ask       | click_bottom_navigation_questions |       6 | view_lab_tap                      |       5 | click_bottom_navigation_profile   |       4 |
-- | 43 | view_friendplus_tap               | view_questions_tap                |       3 | launch_app                        |       1 | $session_end                      |       1 |
-- | 44 | click_notice                      | click_notice_detail               |       1 | –                                 |       – | –                                 |         |



-- WITH ordered_events AS (
--     SELECT
--         session_id,
--         event_datetime,
--         event_key,
--         LEAD(event_key) OVER (PARTITION BY session_id ORDER BY event_datetime) AS next_event_key
--     FROM final.hackle_events
-- ),
-- transition_counts AS (
--     SELECT
--         event_key,
--         next_event_key,
--         COUNT(*) AS transition_count
--     FROM ordered_events
--     WHERE next_event_key IS NOT NULL
--     GROUP BY event_key, next_event_key
-- ),
-- ranked AS (
--     SELECT
--         event_key,
--         next_event_key,
--         transition_count,
--         ROW_NUMBER() OVER (PARTITION BY event_key ORDER BY transition_count DESC) AS rnk
--     FROM transition_counts
-- )
-- SELECT
--     event_key,
--     MAX(CASE WHEN rnk = 1 THEN next_event_key END) AS next_event_1,
--     MAX(CASE WHEN rnk = 1 THEN transition_count END) AS count_1,
--     MAX(CASE WHEN rnk = 2 THEN next_event_key END) AS next_event_2,
--     MAX(CASE WHEN rnk = 2 THEN transition_count END) AS count_2,
--     MAX(CASE WHEN rnk = 3 THEN next_event_key END) AS next_event_3,
--     MAX(CASE WHEN rnk = 3 THEN transition_count END) AS count_3
-- FROM ranked
-- WHERE rnk <= 3
-- GROUP BY event_key
-- ORDER BY count_1 DESC;


-- --------------------------------------------------------------------------------


-- 한 세션 내에서 어떤 이벤트 순서로 발생하는지 확인(evnet_flow)

-- SELECT
--     session_id,
--     GROUP_CONCAT(event_key ORDER BY event_datetime SEPARATOR ', ') AS event_flow
-- FROM final.hackle_events
-- GROUP BY session_id;


-- --------------------------------------------------------------------------------


-- `launch_app → $session_start` or `$session_start → launch_app` 이벤트의 후속 이벤트로 뭐가 제일 잘 나오는지

-- WITH ordered AS (
--     SELECT
--         session_id,
--         event_datetime,
--         event_key,
--         ROW_NUMBER() OVER (PARTITION BY session_id ORDER BY event_datetime) AS rn
--     FROM final.hackle_events
-- ),

-- -- 세션별로 launch_app 또는 $session_start 이후의 첫 번째 이벤트 찾기
-- flow_after_start AS (
--     SELECT
--         o1.session_id,
--         o1.event_key AS start_event,
--         o2.event_key AS next_event,
--         o2.event_datetime AS next_time
--     FROM ordered o1
--     JOIN ordered o2
--       ON o1.session_id = o2.session_id
--      AND o2.rn = o1.rn + 1  -- 바로 다음 이벤트
--     WHERE o1.event_key IN ('launch_app', '$session_start')
-- )

-- -- 전체 세션을 합쳐 다음 이벤트의 빈도수 카운트
-- SELECT
--     next_event,
--     COUNT(*) AS occur_count,
--     ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS ratio_pct
-- FROM flow_after_start
-- WHERE next_event IS NOT NULL
-- GROUP BY next_event
-- ORDER BY occur_count DESC
-- LIMIT 20;


-- --------------------------------------------------------------------------------

-- 해당 서비스의 결제율을 올리는 것이 주 목표
-- 결제한 사람들의 사용 패턴을 알아보자
-- 고유 `session_id`들의 `event_key`를 `event_time`시간 순으로 정렬해도 각 이벤트의 후속 이벤트들은 자유분방하게 나타나져있음
-- 그러면, 고유 `session_id`들이 보유 중인, `event_key`들 중 각 퍼널에 단계에 해당되는 이벤트 키들이 포함된, 고유 `session_id`들만 각 퍼널 단계에 넣어서 고유 `session_id` 가 몇 개인지 파악해보면,
-- 어느 단계에서 이탈률이 가장 심한지, 그리고 해당 퍼널층의 `session_id`들의 이벤트 퍼널들도 분석 할 수 있지 않을까?

-- 결제까지 이뤄진 세션들의 이벤트 플로우 패턴 알아보기
-- 결제까지 이어지는 퍼널 분석

-- 1번 이벤트 키를 보유한 고유 session_id 확보
-- 1번과 2번 이벤트 키를 보유한 고유 session_id 확보,
-- 1, 2, 3번 이벤트 키를 보유한 고유 session_id 확보
-- 1, 2, 3, 4 이벤트 키를 보유한 고유 session_id 확보

-- 1. 앱 실행 ($session_start, launch_app)
-- 2. 서비스 이용 (click_question_start, complete_question)
-- 3. 결제 화면 진입 (view_shop)
-- 4. 결제 완료 (complete_purchase)


-- WITH
-- -- 세션별 이벤트 목록 정리
-- session_events AS (
--     SELECT
--         session_id,
--         GROUP_CONCAT(DISTINCT event_key ORDER BY event_datetime ASC) AS event_list
--     FROM final.hackle_events
--     WHERE session_id IS NOT NULL
--     GROUP BY session_id
-- ),

-- -- 각 세션이 퍼널 단계 이벤트를 포함하는지 여부 계산
-- funnel_flags AS (
--     SELECT
--         session_id,
--         event_list,

--         -- Step1: 앱 실행
--         CASE
--             WHEN event_list LIKE '%session_start%' OR event_list LIKE '%launch_app%' THEN 1
--             ELSE 0
--         END AS has_app_start,

--         -- Step2: 서비스 이용
--         CASE
--             WHEN (event_list LIKE '%click_question_start%' OR event_list LIKE '%complete_question%')
--                  AND (event_list LIKE '%session_start%' OR event_list LIKE '%launch_app%')
--             THEN 1 ELSE 0
--         END AS has_use_service,

--         -- Step3: 결제 화면 진입
--         CASE
--             WHEN (event_list LIKE '%view_shop%')
--                  AND (event_list LIKE '%click_question_start%' OR event_list LIKE '%complete_question%')
--                  AND (event_list LIKE '%session_start%' OR event_list LIKE '%launch_app%')
--             THEN 1 ELSE 0
--         END AS has_view_shop,

--         -- Step4: 결제 완료
--         CASE
--             WHEN (event_list LIKE '%complete_purchase%')
--                  AND (event_list LIKE '%view_shop%')
--                  AND (event_list LIKE '%click_question_start%' OR event_list LIKE '%complete_question%')
--                  AND (event_list LIKE '%session_start%' OR event_list LIKE '%launch_app%')
--             THEN 1 ELSE 0
--         END AS has_complete_purchase
--     FROM session_events
-- ),

-- -- 각 퍼널 단계별 세션 수 집계
-- funnel_counts AS (
--     SELECT
--         COUNT(DISTINCT CASE WHEN has_app_start = 1 THEN session_id END) AS step1_sessions,
--         COUNT(DISTINCT CASE WHEN has_use_service = 1 THEN session_id END) AS step2_sessions,
--         COUNT(DISTINCT CASE WHEN has_view_shop = 1 THEN session_id END) AS step3_sessions,
--         COUNT(DISTINCT CASE WHEN has_complete_purchase = 1 THEN session_id END) AS step4_sessions
--     FROM funnel_flags
-- )

-- -- 퍼널 단계별 결과 출력
-- SELECT
--     '1. 앱 실행' AS step_name,
--     step1_sessions AS session_count,
--     100.00 AS conversion_rate
-- FROM funnel_counts

-- UNION ALL
-- SELECT
--     '2. 서비스 이용',
--     step2_sessions,
--     ROUND(step2_sessions / step1_sessions * 100, 2)
-- FROM funnel_counts

-- UNION ALL
-- SELECT
--     '3. 결제 화면 진입',
--     step3_sessions,
--     ROUND(step3_sessions / step1_sessions * 100, 2)
-- FROM funnel_counts

-- UNION ALL
-- SELECT
--     '4. 결제 완료',
--     step4_sessions,
--     ROUND(step4_sessions / step1_sessions * 100, 2)
-- FROM funnel_counts;


-- --------------------------------------------------------------------------------


-- 해당 서비스의 결제율을 올리는 것이 주 목표
-- 결제가 이루어지는 퍼널 구조를 알아보자

-- 현재 device_id와 user_id, session_id가 엉켜있는 상태
-- device_id에 여러 개의 user_id 존재 확인
-- user_id에 여러 개의 session_id 존재 확인
-- user_id에 여러 개의 device_id 존재 확인

-- ## device_id 기준으로 볼 때?
-- 1. 디바이스 기준으로 확인한다
-- 2. 디바이스 아이디 테이블의 user_id가 결측인 데이터는 삭제
-- 3. 각 유저 아이디에 디바이스 아이디가 2건 이상 존재 시 해당 데이터는 삭제 (전체 데이터 대비 얼마 없음)
-- 4. 처리한 디바이스 아이디들 중 유저 아이디가 보유중인 session_id 확인
-- 5. 결제 완료 이벤트 키가 있는 session_id를 보유한 user_id를 보유중인 device_id를 결제 퍼널에 사용한다

-- ## user_id 기준으로 볼 때?
-- 1. session_id와 user_id가 엮여있는 테이블에서 user_id 별로 보유중인 session_id를 확인
-- 2. 해당 session_id들이 보유중인 event_key들을 event_datetime 순으로 나열해서 각 user_id별 한 줄로 리스트화
-- 3. 해당 user_id들의 event_flow를 살펴본 후 complete_purchase를 보유 중인 event_flow 유저들을 결제 퍼널 구조 확인에 사용

-- 일단 어떤 기준으로 고유 유저들을 정리하던간에 무조건 확인해봐야하는 점
-- ### complete_purchase 를 보유 중인 고유 유저들이 몇 명인지 확인
-- 그래야지 중간 퍼널 과정에서 놓친 이벤트 키를 보강할 수 있음

In [ ]:
SELECT * FROM final.hackle_properties;



-- 테이블 전체 행 수 (525,350)

-- SELECT
-- 	COUNT(*)
-- FROM final.hackle_properties;

-- ------------------------------------------------------------

-- 고유 device_id 수 (251,720)

-- SELECT
-- 	COUNT(DISTINCT device_id)
-- FROM final.hackle_properties;


-- 고유 user_id 수 (327,381)
-- SELECT
-- 	COUNT(DISTINCT user_id)
-- FROM final.hackle_properties;


-- 전체 데이터 기준
SELECT
    COUNT(*) AS total_rows,                         -- 전체 행 수
    COUNT(DISTINCT device_id) AS total_device_ids,  -- 전체 고유 device_id
    COUNT(DISTINCT user_id) AS total_user_ids       -- 전체 고유 user_id
FROM final.hackle_properties;


-- ------------------------------------------------------------

-- user_id가 NULL 또는 빈 문자열('')인 행 수
-- session_id가 NULL 또는 빈 문자열('')인 행 수
-- 위 두 조건을 동시에 만족하지 않는 유효한 device_id 행 수
-- 그리고 각 비율(%)

-- | 항목                                    | 값            | 비율(%)     | 해석                               |
-- | 전체 행 수                               |   525,350건   |   100.00   | hackle_properties 테이블 전체 행 수     |
-- | user_id 결측 또는 빈 값                   |   82,255건    |   15.66%   | 약 15.7%의 레코드는 user_id가 없거나 비어 있음 |
-- | session_id 결측 또는 빈 값                |   0건         |   0.00%    | 모든 레코드에 session_id는 존재함          |
-- | 유효(user_id + session_id 둘 다 존재) 행 수 |  443,095건   |   84.34%   | 실제 분석에 사용할 수 있는 데이터 비율           |


-- SELECT
--     COUNT(*) AS total_rows,

--     -- user_id 결측/빈값
--     SUM(user_id IS NULL OR user_id = '') AS null_userid_rows,
--     ROUND(SUM(user_id IS NULL OR user_id = '') / COUNT(*) * 100, 2) AS null_userid_ratio,

--     -- session_id 결측/빈값
--     SUM(session_id IS NULL OR session_id = '') AS null_sessionid_rows,
--     ROUND(SUM(session_id IS NULL OR session_id = '') / COUNT(*) * 100, 2) AS null_sessionid_ratio,

--     -- user_id, session_id 둘 다 유효한 행
--     SUM(user_id IS NOT NULL AND user_id <> '' AND session_id IS NOT NULL AND session_id <> '') AS valid_rows,
--     ROUND(SUM(user_id IS NOT NULL AND user_id <> '' AND session_id IS NOT NULL AND session_id <> '') / COUNT(*) * 100, 2) AS valid_ratio
-- FROM final.hackle_properties;


-- ------------------------------------------------------------


-- user_id가 있는 device_id 데이터만 대상으로
-- 고유 user_id 당 device_id가 여러 개인 경우를 파악하고,
-- 그 중복 이력이 있는 user_id를 보유한 고유 device_id 행을 제거했을 때 데이터가 얼마나 줄어드는지 확인하기

-- | 항목                                             | 값      | 비율(%)  | 의미                                      |
-- | total_device_id_count(user_id NULL제외)          | 243,364 | 100.00 | `user_id` 존재하는 전체 고유 `device_id` 수      |
-- | device_id_linked_to_multi_user                  | 5,742   | 2.36   | 여러 기기를 사용한 `user_id`와 연결된 `device_id` 수 |
-- | remaining_device_id_count                       | 237,622 | 97.64  | 중복 유저 제거 후 남은 `device_id` 수             |
-- | remaining_ratio                                 | 97.64   | -      | 전체 중 유지된 비율                             |


-- WITH valid_data AS (
--     -- ① user_id 있는 데이터만 남김
--     SELECT device_id, user_id
--     FROM final.hackle_properties
--     WHERE user_id IS NOT NULL AND user_id <> ''
-- ),

-- multi_device_user AS (
--     -- ② user_id별 device_id 수 계산
--     SELECT user_id
--     FROM valid_data
--     GROUP BY user_id
--     HAVING COUNT(DISTINCT device_id) > 1   -- 여러 device 사용 이력
-- ),

-- device_summary AS (
--     -- ③ 각 user_id가 속한 device_id 행 (중복 user 포함)
--     SELECT DISTINCT device_id,
--            CASE WHEN user_id IN (SELECT user_id FROM multi_device_user) THEN 1 ELSE 0 END AS is_multi_device_user
--     FROM valid_data
-- )

-- -- ④ 결과 비교 출력
-- SELECT
--     COUNT(*) AS total_device_id_count,
--     SUM(is_multi_device_user) AS device_id_linked_to_multi_user,
--     ROUND(SUM(is_multi_device_user) / COUNT(*) * 100, 2) AS multi_user_ratio,
--     COUNT(*) - SUM(is_multi_device_user) AS remaining_device_id_count,
--     ROUND((COUNT(*) - SUM(is_multi_device_user)) / COUNT(*) * 100, 2) AS remaining_ratio
-- FROM device_summary;


-- ------------------------------------------------------------

-- 지금까지의 과정 정리
-- --------------------------------------------
-- ### 주요 결과
-- 테이블 전체 행: 525,350건
-- user_id 결측: 82,255건 (15.66%)
-- 유효(user_id 존재) device_id 데이터: 443,095건 (84.34%)
--
-- user_id 존재 고유 device_id 수: 243,364건
-- 다중 device user_id 연결 고유 device_id: 5,742건 (2.36%)
-- 제거 후 남은 고유 device_id: 237,622건 (97.64%)
--
-- 결론
-- 약 97.6%의 device_id는 1 user_id와 1:1 관계로 유지됨
-- 퍼널 분석용 기준 집합으로 활용 가능
-- --------------------------------------------

-- ------------------------------------------------------------


-- device_id_summary 생성
-- 기준
-- device_id 중 user_id가 존재하지 않는 행 제외,
-- user_id가 여러 개의 device_id 보유 시 행 제외
-- user_id 별 session_id를 시간 순으로 이어붙임 세션블록이 달라도 한 줄로 이어붙임
-- 세션블록을 이어 붙일 땐 `/` 로 구분되고 한 세션 안에서 이뤄지는 구분자는 `,`로 구분됨


-- CREATE OR REPLACE VIEW final.device_id_summary AS
-- WITH valid_properties AS (
--     -- user_id가 존재하는 데이터만 사용
--     SELECT device_id, user_id, session_id
--     FROM final.hackle_properties
--     WHERE user_id IS NOT NULL AND user_id <> ''
-- ),
-- multi_device_user AS (
--     -- 여러 device_id를 사용한 user_id 제거 대상 식별
--     SELECT user_id
--     FROM valid_properties
--     GROUP BY user_id
--     HAVING COUNT(DISTINCT device_id) > 1
-- ),
-- clean_properties AS (
--     -- 다중 기기 user 제거 후 남은 device-user-session 데이터
--     SELECT *
--     FROM valid_properties
--     WHERE user_id NOT IN (SELECT user_id FROM multi_device_user)
-- ),
-- session_event_flow AS (
--     -- 세션 단위 이벤트 흐름 생성 (이벤트 시간순 정렬)
--     SELECT
--         session_id,
--         GROUP_CONCAT(event_key ORDER BY event_datetime ASC SEPARATOR ',') AS session_flow,
--         MIN(event_datetime) AS session_start_time
--     FROM final.hackle_events
--     GROUP BY session_id
-- ),
-- device_event_flow AS (
--     -- device_id 단위로 모든 세션을 시간 순으로 병합 (' / '로 구분)
--     SELECT
--         cp.device_id,
--         cp.user_id,
--         GROUP_CONCAT(DISTINCT cp.session_id ORDER BY sf.session_start_time ASC SEPARATOR ',') AS session_list,
--         COUNT(DISTINCT cp.user_id) AS user_id_count,
--         COUNT(DISTINCT cp.session_id) AS session_count,
--         GROUP_CONCAT(sf.session_flow ORDER BY sf.session_start_time ASC SEPARATOR ' / ') AS event_flow
--     FROM clean_properties cp
--     JOIN session_event_flow sf ON cp.session_id = sf.session_id
--     GROUP BY cp.device_id, cp.user_id
-- )
-- -- 최종 결과 출력 (행번호 포함)
-- SELECT
--     ROW_NUMBER() OVER (ORDER BY device_id) AS num,
--     device_id,
--     user_id,
--     user_id_count,
--     session_count AS user_session_count,
--     session_list,
--     event_flow
-- FROM device_event_flow;


In [ ]:
-- -----------------------------------------------------------
-- user_id 정제 전후 데이터 행 수 비교 (삭제될 행 수 확인)
-- 전체 행(525,350) 중 191,259행 삭제 → 334,091행 생존
-- -----------------------------------------------------------
-- SELECT
--     COUNT(*) AS total_rows,   -- 전체 행 수
--     SUM(
--         user_id IS NULL
--         OR TRIM(user_id) = ''
--         OR user_id REGEXP '[^0-9]'
--     ) AS to_be_deleted_rows,  -- 제거 대상 (문자난수 / 공란 / NULL)
--     COUNT(*) - SUM(
--         user_id IS NULL
--         OR TRIM(user_id) = ''
--         OR user_id REGEXP '[^0-9]'
--     ) AS remaining_rows       -- 필터링 후 남는 행 수
-- FROM final.hackle_properties;


-- -----------------------------------------------------------
-- user_id 정제 후 남은 데이터 기준의 고유 device_id / user_id 수
-- 1차 정제(user_id 문자 난수 & 공란 & null값 처리) 후 고유 device_id 수 : 233,892
-- 1차 정제(user_id 문자 난수 & 공란 & null값 처리) 후 고유 user_id 수 : 230,853
-- -----------------------------------------------------------
-- SELECT
--     COUNT(DISTINCT device_id) AS unique_device_id_after_cleaning,  -- 고유 device_id 수
--     COUNT(DISTINCT user_id) AS unique_user_id_after_cleaning        -- 고유 user_id 수
-- FROM final.hackle_properties
-- WHERE user_id IS NOT NULL
--   AND TRIM(user_id) <> ''
--   AND user_id REGEXP '^[0-9]+$';   -- 숫자형 user_id만 남김

-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- STEP 1: 정제된 데이터만 남긴 임시 테이블 생성
-- -----------------------------------------------------------

-- CREATE TABLE final.step1 AS
-- SELECT
--     session_id,
--     user_id,
--     osname,
--     device_id
-- FROM final.hackle_properties
-- WHERE user_id IS NOT NULL
--   AND TRIM(user_id) <> ''
--   AND user_id REGEXP '^[0-9]+$';  -- 숫자형 user_id만 남김

-- -----------------------------------------------------------


-- -----------------------------------------------------------
-- [STEP1 요약 분석] user_id ↔ device_id 관계 전체 요약
-- -----------------------------------------------------------

-- | 지표                              | 설명                             |           결과 |
-- | **total_users**                 | 정제된 유저 총수 (`user_id` 고유값)    | **230,853명** |
-- | **users_with_multiple_devices** | 2개 이상의 `device_id`를 사용한 유저 수 |   **2,555명** |
-- | **multiple_device_ratio_pct**   | 여러 기기를 사용한 유저 비율             |    **1.11%** |
-- | **avg_device_per_user**         | 유저당 평균 디바이스 개수               |   **1.013대** |
-- | **users_with_1_device**         | 한 대의 기기만 사용한 유저 수            | **228,298명** |
-- | **users_with_2_devices**        | 두 대의 기기를 사용한 유저 수            |   **2,300명** |
-- | **users_with_3plus_devices**    | 세 대 이상을 사용한 유저 수             |     **255명** |



-- WITH user_device_stats AS (
--     SELECT
--         user_id,
--         COUNT(*) AS total_rows,                    -- 해당 user_id가 찍힌 전체 행 수
--         COUNT(DISTINCT device_id) AS device_count  -- 해당 user_id의 고유 device 수
--     FROM final.step1
--     GROUP BY user_id
-- )

-- -- 최종 요약 출력
-- SELECT
--     -- 전체 user_id 수
--     COUNT(*) AS total_users,

--     -- 여러 기기를 사용한 user 수
--     SUM(device_count > 1) AS users_with_multiple_devices,

--     -- 여러 기기 사용 비율 (%)
--     ROUND(100 * SUM(device_count > 1) / COUNT(*), 2) AS multiple_device_ratio_pct,

--     -- user당 평균 device 수
--     ROUND(AVG(device_count), 3) AS avg_device_per_user,

--     -- device_id 1개만 사용한 user 수
--     SUM(device_count = 1) AS users_with_1_device,

--     -- device_id 2개 사용한 user 수
--     SUM(device_count = 2) AS users_with_2_devices,

--     -- device_id 3개 이상 사용한 user 수
--     SUM(device_count >= 3) AS users_with_3plus_devices

-- FROM user_device_stats;


-- -------------------------------------------------------

-- -----------------------------------------------------------
-- 여러 대를 거쳐간(user_id 당 device_id 수 > 1) user_id와 관련된 행 통계

-- | 구분                                | 설명                                   |           결과 |
-- | **rows_to_delete**                | 여러 device를 사용한 user_id가 포함된 전체 행 수 |   **7,296행** |
-- | **device_ids_to_delete**          | 해당 user_id들과 연결된 고유 device_id 수    |   **5,657개** |
-- | **user_ids_affected**             | 여러 device를 사용한 user_id 수           |   **2,555명** |
-- | **total_rows**                    | 현재 step1 전체 행 수                    | **334,091행** |
-- | **remaining_rows_after_deletion** | 삭제 후 남게 되는 행 수                     | **326,795행** |
-- | **delete_ratio_pct**              | 전체 중 삭제될 비율                        |    **2.18%** |

-- -----------------------------------------------------------
-- WITH multi_users AS (
--     SELECT user_id
--     FROM final.step1
--     GROUP BY user_id
--     HAVING COUNT(DISTINCT device_id) > 1
-- )
-- SELECT
--     -- 삭제 대상 총 행 수 (해당 user_id를 가진 모든 행)
--     (SELECT COUNT(*) FROM final.step1 WHERE user_id IN (SELECT user_id FROM multi_users)) AS rows_to_delete,

--     -- 삭제 대상에서 나타나는 고유 device_id 수
--     (SELECT COUNT(DISTINCT device_id) FROM final.step1 WHERE user_id IN (SELECT user_id FROM multi_users)) AS device_ids_to_delete,

--     -- 영향을 받는 user_id 수 (multi_users 건수)
--     (SELECT COUNT(DISTINCT user_id) FROM final.step1 WHERE user_id IN (SELECT user_id FROM multi_users)) AS user_ids_affected,

--     -- 전체 행 수 (참고)
--     (SELECT COUNT(*) FROM final.step1) AS total_rows,

--     -- 삭제 후 남는 행 수
--     (SELECT COUNT(*) FROM final.step1) - (SELECT COUNT(*) FROM final.step1 WHERE user_id IN (SELECT user_id FROM multi_users)) AS remaining_rows_after_deletion,

--     -- 삭제 비율 (%)
--     ROUND(
--       100 * (SELECT COUNT(*) FROM final.step1 WHERE user_id IN (SELECT user_id FROM multi_users))
--       / (SELECT COUNT(*) FROM final.step1)
--     , 2) AS delete_ratio_pct;



-- ------------------------------------------------------------
-- STEP 2 : 여러 device를 거친 user_id 전체 제거 후 새 테이블 생성
-- -----------------------------------------------------------
-- CREATE TABLE final.step2 AS
-- SELECT *
-- FROM final.step1
-- WHERE user_id NOT IN (
--     SELECT user_id
--     FROM (
--         SELECT user_id
--         FROM final.step1
--         GROUP BY user_id
--         HAVING COUNT(DISTINCT device_id) > 1
--     ) AS multi_device_users
-- );

-- -----------------------------------------------------------
-- step2 테이블의 데이터 규모 요약
-- | 구분                                              | 설명                      |          결과 |
-- | **전체 행 수 (`total_rows`)**                       | step2 테이블 내 전체 데이터 수 | **326,795** |
-- | **고유 `device_id` 수 (`unique_device_id_count`)** | 중복 제거된 고유 디바이스 개수    | **228,251** |
-- | **고유 `user_id` 수 (`unique_user_id_count`)**     | 중복 제거된 고유 유저 개수      | **228,298** |


-- SELECT
--     COUNT(*) AS total_rows,                  -- 전체 행 수
--     COUNT(DISTINCT device_id) AS unique_device_id_count,  -- 고유 device_id 수
--     COUNT(DISTINCT user_id) AS unique_user_id_count       -- 고유 user_id 수
-- FROM final.step2;
-- -----------------------------------------------------------



-- -----------------------------------------------------------
-- 여러 user_id를 가진 device_id 관련 전체 행/비율 요약
-- | 구분                              | 설명                          |        결과 |
-- | **devices_with_multiple_users** | 여러 user_id가 연결된 device_id 수 |   **46개** |
-- | **device_ratio_pct**            | 전체 device_id 중 차지 비율        | **0.02%** |
-- | **rows_with_multiple_users**    | 해당 device_id들이 포함된 전체 행 수   |  **125행** |
-- | **row_ratio_pct**               | 전체 데이터 중 차지 비율              | **0.04%** |



-- WITH multi_device AS (
--     SELECT device_id
--     FROM final.step2
--     GROUP BY device_id
--     HAVING COUNT(DISTINCT user_id) > 1
-- )
-- SELECT
--     -- 여러 user_id 가진 device_id 수
--     (SELECT COUNT(*) FROM multi_device) AS devices_with_multiple_users,

--     -- 전체 device_id 대비 비율 (%)
--     ROUND(
--         100 * (SELECT COUNT(*) FROM multi_device)
--         / (SELECT COUNT(DISTINCT device_id) FROM final.step2),
--         2
--     ) AS device_ratio_pct,

--     -- 해당 device_id들이 포함된 전체 행 수
--     (SELECT COUNT(*) FROM final.step2 WHERE device_id IN (SELECT device_id FROM multi_device)) AS rows_with_multiple_users,

--     -- 전체 행 대비 비율 (%)
--     ROUND(
--         100 * (SELECT COUNT(*) FROM final.step2 WHERE device_id IN (SELECT device_id FROM multi_device))
--         / (SELECT COUNT(*) FROM final.step2),
--         2
--     ) AS row_ratio_pct;

-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- STEP 3 : 여러 user_id를 거친 device_id 전체 제거 후 새 테이블 생성

-- CREATE TABLE final.step3 AS
-- SELECT *
-- FROM final.step2
-- WHERE device_id NOT IN (
--     SELECT device_id
--     FROM (
--         SELECT device_id
--         FROM final.step2
--         GROUP BY device_id
--         HAVING COUNT(DISTINCT user_id) > 1
--     ) AS multi_user_devices
-- );

-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- STEP3 테이블 데이터 규모 요약(1:1 매핑 완성 → 이제 세션 정리해주면 끝)
-- | 구분                                              | 설명                 |          결과 |
-- | **전체 행 수 (`total_rows`)**                       | 정제 후 남은 전체 데이터 수 | **326,670** |
-- | **고유 `device_id` 수 (`unique_device_id_count`)** | 중복 제거된 디바이스 수    | **228,205** | ==
-- | **고유 `user_id` 수 (`unique_user_id_count`)**     | 중복 제거된 유저 수      | **228,205**  | ==

-- SELECT
--     COUNT(*) AS total_rows,                   -- 전체 행 수
--     COUNT(DISTINCT device_id) AS unique_device_id_count,  -- 고유 device_id 수
--     COUNT(DISTINCT user_id) AS unique_user_id_count       -- 고유 user_id 수
-- FROM final.step3;

-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- 여러 개의 session_id를 가진 user_id 찾기 (중복 세션id는 아직 처리를 안했으니 제외)

-- | 구분                               | 설명                                  |        결과 |
-- | **users_with_multiple_sessions** | 서로 다른 `session_id`를 2개 이상 가진 유저 수 |  **186명** |
-- | **ratio_pct**                    | 전체 유저 대비 비율                         | **0.08%** |



-- WITH user_session_stats AS (
--     SELECT
--         user_id,
--         COUNT(DISTINCT session_id) AS unique_session_count  -- 중복 제거된 세션 개수
--     FROM final.step3
--     WHERE session_id IS NOT NULL AND TRIM(session_id) <> ''
--     GROUP BY user_id
-- )
-- SELECT
--     COUNT(*) AS users_with_multiple_sessions,  -- 여러 세션 보유한 user 수
--     ROUND(100 * COUNT(*) / (SELECT COUNT(DISTINCT user_id) FROM final.step3), 2) AS ratio_pct
-- FROM user_session_stats
-- WHERE unique_session_count > 1;
-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- STEP 4 : (user_id, device_id, session_id) 완전 중복 제거 버전


-- CREATE TABLE final.step4 AS
-- SELECT
--     session_id,
--     user_id,
--     osname,
--     device_id
-- FROM final.step3
-- GROUP BY
--     session_id,
--     user_id,
--     device_id,
--     osname;  -- 완전히 동일한 조합의 중복 행 제거
-- -----------------------------------------------------------



-- -----------------------------------------------------------
-- step4 데이터 정제 결과 요약
-- | 구분                    | 설명                           |          결과 |
-- | **total_rows**        | 전체 행 수 (중복 제거 후 남은 실제 데이터 수) | **228,391** |
-- | **unique_device_id**  | 고유한 디바이스 수                   | **228,205** |
-- | **unique_user_id**    | 고유한 유저 수                     | **228,205** |
-- | **unique_session_id** | 고유한 세션 수                     | **228,390** |

-- SELECT
--     COUNT(*) AS total_rows,
--     COUNT(DISTINCT device_id) AS unique_device_id,
--     COUNT(DISTINCT user_id) AS unique_user_id,
--     COUNT(DISTINCT session_id) AS unique_session_id
-- FROM final.step4;
-- -----------------------------------------------------------



-- -----------------------------------------------------------
-- -----------------------------------------------------------
-- [최종 단계] device_user_id_summary 테이블 생성


-- CREATE TABLE final.device_user_id_summary AS
-- SELECT
--     ROW_NUMBER() OVER (ORDER BY device_id) AS num,   -- 순번(PK 역할)
--     device_id,
--     user_id,
--     ANY_VALUE(osname) AS osname,                     -- 같은 디바이스 내 OS 동일하므로 하나만 선택
--     GROUP_CONCAT(DISTINCT session_id ORDER BY session_id SEPARATOR ',') AS session_id,  -- 세션 목록
--     COUNT(DISTINCT session_id) AS session_count      -- 세션 개수
-- FROM final.step4
-- GROUP BY device_id, user_id;
-- -----------------------------------------------------------


-- -----------------------------------------------------------
-- 최종 테이블 검증
-- | 구분                   | 설명         |          결과 |
-- | **total_rows**       | 테이블 전체 행 수 | **228,205** |
-- | **unique_device_id** | 고유 디바이스 수  | **228,205** |
-- | **unique_user_id**   | 고유 유저 수    | **228,205** |


-- SELECT
--     COUNT(*) AS total_rows,
--     COUNT(DISTINCT device_id) AS unique_device_id,
--     COUNT(DISTINCT user_id) AS unique_user_id
-- FROM final.device_user_id_summary;


In [ ]:
SELECT * FROM final.device_user_id_summary;
-- ----------------------------------------

-- -----------------------------------------------------------
-- session_count == 1 인 유저들의 evnet_flow 확인 테이블 제작

-- -----------------------------------------------------------
-- hackle_events 테이블 성능 최적화를 위한 인덱스 생성
-- CREATE INDEX idx_hackle_events_session_datetime
-- ON final.hackle_events (session_id, event_datetime);
-- -----------------------------------------------------------

-- -----------------------------------------------------------
-- ------------------------------------------------------------
-- [목적]
-- device_user_id_summary 테이블의 기존 컬럼 유지
-- + session_count_1_log의 event_flow 컬럼을 session_id 기준으로 병합
-- 결과를 session_count_1_log_e 테이블로 생성
-- ------------------------------------------------------------
-- 새 테이블 생성
-- CREATE TABLE final.session_count_1_log_e AS
-- SELECT
--     dus.num,              -- 기존 PK (순번)
--     dus.device_id,
--     dus.user_id,
--     dus.osname,
--     dus.session_id,
--     dus.session_count,
--     scl.event_flow        -- 새로 추가되는 event_flow 컬럼
-- FROM final.device_user_id_summary AS dus
-- LEFT JOIN final.session_count_1_log AS scl
--     ON dus.session_id = scl.session_id
-- ORDER BY dus.user_id;